# 🧠 Lab 1: Agent vs Workflow
## Module 0 - Understanding the Fundamental Difference

**Duration:** 15 minutes

**Objectives:**
- Build a deterministic workflow for wire transfer validation
- Build an agent for fraud investigation
- Understand when to use each approach

**Banking Scenario:** Wire transfer processing vs Fraud investigation

---

In [ ]:
!pip install openai -q

In [ ]:
import os
import json

# =============================================================================
# GOOGLE COLAB SETUP
# =============================================================================
# Add these secrets in Colab (click 🔑 icon in left sidebar):
#   - AZURE_OPENAI_KEY: Your API key
#   - AZURE_OPENAI_ENDPOINT: Your endpoint (https://xxx.openai.azure.com/)
#   - AZURE_OPENAI_DEPLOYMENT: Your model deployment name
# =============================================================================

DEMO_MODE = False
client = None
MODEL_NAME = "gpt-4o"

try:
    from google.colab import userdata
    AZURE_OPENAI_KEY = userdata.get('AZURE_OPENAI_KEY')
    AZURE_OPENAI_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')
    
    try:
        MODEL_NAME = userdata.get('AZURE_OPENAI_DEPLOYMENT')
    except:
        pass
    
    if AZURE_OPENAI_KEY and AZURE_OPENAI_ENDPOINT:
        if not AZURE_OPENAI_ENDPOINT.startswith('http'):
            AZURE_OPENAI_ENDPOINT = 'https://' + AZURE_OPENAI_ENDPOINT
        print("✅ Credentials loaded from Colab secrets")
        print(f"   Model: {MODEL_NAME}")
    else:
        raise ValueError("Missing credentials")
except Exception as e:
    print(f"⚠️ Colab secrets not found: {e}")
    print("   Running in DEMO MODE - API calls will be simulated")
    DEMO_MODE = True

if not DEMO_MODE:
    from openai import AzureOpenAI
    client = AzureOpenAI(
        api_key=AZURE_OPENAI_KEY,
        api_version="2024-06-01",
        azure_endpoint=AZURE_OPENAI_ENDPOINT
    )
    print("✅ Azure OpenAI client ready")

## Part 1: WORKFLOW - Deterministic Wire Transfer Validation

A workflow has **predefined steps** where the **code decides** what happens next.

**Key characteristics:**
- Deterministic: Same input → Same output
- Fast: Milliseconds execution
- No LLM required: Pure code logic

In [ ]:
# Simulated sanctions list (in production: call OFAC API)
SANCTIONS_LIST = ["BLOCKED_ENTITY_1", "SANCTIONED_CORP", "RESTRICTED_BANK"]

def workflow_wire_transfer(amount: float, balance: float, recipient: str) -> dict:
    """
    WORKFLOW: Code decides every step
    Same input ALWAYS produces same output
    """
    print(f"\n🔄 WORKFLOW: Processing wire transfer")
    print(f"   Amount: ${amount:,.2f}")
    print(f"   Balance: ${balance:,.2f}")
    print(f"   Recipient: {recipient}")
    
    # Step 1: Validate amount (CODE DECIDES)
    print("\n   Step 1: Validating amount...")
    if amount <= 0:
        return {"status": "rejected", "reason": "Invalid amount", "step": 1}
    print("   ✅ Amount valid")
    
    # Step 2: Check balance (CODE DECIDES)
    print("   Step 2: Checking balance...")
    if amount > balance:
        return {"status": "rejected", "reason": "Insufficient funds", "step": 2}
    print("   ✅ Sufficient funds")
    
    # Step 3: Sanctions screening (CODE DECIDES)
    print("   Step 3: Sanctions screening...")
    if recipient.upper() in [s.upper() for s in SANCTIONS_LIST]:
        return {"status": "blocked", "reason": "Sanctions match", "step": 3}
    print("   ✅ No sanctions match")
    
    # Step 4: Execute transfer (CODE DECIDES)
    print("   Step 4: Executing transfer...")
    new_balance = balance - amount
    print(f"   ✅ Transfer complete. New balance: ${new_balance:,.2f}")
    
    return {
        "status": "approved",
        "amount": amount,
        "new_balance": new_balance,
        "recipient": recipient
    }

In [ ]:
# Test the workflow with different scenarios
print("="*60)
print("TEST 1: Valid transfer")
result = workflow_wire_transfer(amount=5000, balance=10000, recipient="ACME Corp")
print(f"Result: {result}")

print("\n" + "="*60)
print("TEST 2: Insufficient funds")
result = workflow_wire_transfer(amount=15000, balance=10000, recipient="ACME Corp")
print(f"Result: {result}")

print("\n" + "="*60)
print("TEST 3: Sanctions match")
result = workflow_wire_transfer(amount=5000, balance=10000, recipient="SANCTIONED_CORP")
print(f"Result: {result}")

## Part 2: AGENT - LLM-Driven Fraud Investigation

An agent has **dynamic behavior** where the **LLM decides** what to do next.

**Key characteristics:**
- Non-deterministic: LLM reasoning varies
- Flexible: Can handle novel situations
- Tool-using: LLM chooses which tools to call

In [ ]:
# Define tools the agent can use
fraud_tools = [
    {
        "type": "function",
        "function": {
            "name": "get_transaction_history",
            "description": "Get customer transaction history for analysis",
            "parameters": {
                "type": "object",
                "properties": {
                    "customer_id": {"type": "string", "description": "Customer ID"},
                    "days": {"type": "integer", "description": "Days to look back"}
                },
                "required": ["customer_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_account_changes",
            "description": "Get recent account changes (email, phone, password)",
            "parameters": {
                "type": "object",
                "properties": {
                    "customer_id": {"type": "string", "description": "Customer ID"}
                },
                "required": ["customer_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_device_info",
            "description": "Get device information for account access",
            "parameters": {
                "type": "object",
                "properties": {
                    "customer_id": {"type": "string", "description": "Customer ID"}
                },
                "required": ["customer_id"]
            }
        }
    }
]

def execute_tool(name: str, args: dict) -> dict:
    """Simulate tool execution"""
    if name == "get_transaction_history":
        return {
            "customer_id": args.get("customer_id"),
            "avg_transaction": 2500,
            "max_transaction": 8000,
            "international_transfers": 0,
            "typical_recipients": ["Utility Co", "Grocery Store", "Gas Station"]
        }
    elif name == "get_account_changes":
        return {
            "customer_id": args.get("customer_id"),
            "email_changed": "3 days ago",
            "phone_changed": "3 days ago",
            "password_reset": "2 days ago",
            "address_changed": None
        }
    elif name == "get_device_info":
        return {
            "customer_id": args.get("customer_id"),
            "usual_device": "iPhone 14, San Francisco",
            "current_device": "Android, Lagos Nigeria",
            "new_device": True
        }
    return {"error": "Unknown tool"}

In [ ]:
def run_demo_investigation(alert: str) -> dict:
    """Demo mode: Simulate agent behavior without API calls"""
    print("\n   ⚠️ Running in DEMO MODE")
    print("   Simulating agent investigation...\n")
    
    trace = []
    
    # Iteration 1
    print("   --- Iteration 1 ---")
    args = {"customer_id": "C-789", "days": 30}
    print(f"   🔧 Agent calls: get_transaction_history({args})")
    result = execute_tool("get_transaction_history", args)
    print(f"   📊 Result: {result}")
    trace.append({"tool": "get_transaction_history", "result": result})
    
    # Iteration 2
    print("\n   --- Iteration 2 ---")
    args = {"customer_id": "C-789"}
    print(f"   🔧 Agent calls: get_account_changes({args})")
    result = execute_tool("get_account_changes", args)
    print(f"   📊 Result: {result}")
    trace.append({"tool": "get_account_changes", "result": result})
    
    # Iteration 3
    print("\n   --- Iteration 3 ---")
    args = {"customer_id": "C-789"}
    print(f"   🔧 Agent calls: get_device_info({args})")
    result = execute_tool("get_device_info", args)
    print(f"   📊 Result: {result}")
    trace.append({"tool": "get_device_info", "result": result})
    
    print("\n   ✅ Investigation complete")
    
    recommendation = """**RECOMMENDATION: BLOCK**

**Risk Assessment: HIGH**

**Findings:**
1. Transaction Anomaly: $45,000 is 5.6x higher than max historical ($8,000)
2. Account Takeover Indicators:
   - Email changed 3 days ago
   - Phone changed 3 days ago
   - Password reset 2 days ago
3. Device/Location Anomaly:
   - Usual: iPhone 14, San Francisco
   - Current: Android, Lagos Nigeria (NEW)
4. Zero history of international transfers

**Action:** Block transaction, contact customer, initiate security review.

*[DEMO MODE - Simulated response]*"""
    
    return {"recommendation": recommendation, "trace": trace, "iterations": 4}

In [ ]:
def agent_fraud_investigation(alert: str, max_iterations: int = 5) -> dict:
    """AGENT: LLM decides what to do next"""
    print(f"\n🤖 AGENT: Starting fraud investigation")
    print(f"   Alert: {alert}")
    
    if DEMO_MODE or client is None:
        return run_demo_investigation(alert)
    
    messages = [
        {
            "role": "system",
            "content": """You are a fraud investigation agent at a bank.
When given a fraud alert, investigate by:
1. Gathering relevant information using available tools
2. Analyzing patterns and anomalies
3. Making a recommendation (BLOCK, REVIEW, or ALLOW)

Always explain your reasoning. Be thorough but efficient."""
        },
        {"role": "user", "content": f"Investigate this alert: {alert}"}
    ]
    
    trace = []
    
    try:
        for i in range(max_iterations):
            print(f"\n   --- Iteration {i+1} ---")
            
            response = client.chat.completions.create(
                model=MODEL_NAME,
                messages=messages,
                tools=fraud_tools
            )
            
            msg = response.choices[0].message
            messages.append(msg)
            
            if msg.tool_calls:
                for tc in msg.tool_calls:
                    tool_name = tc.function.name
                    tool_args = json.loads(tc.function.arguments)
                    
                    print(f"   🔧 Agent calls: {tool_name}({tool_args})")
                    result = execute_tool(tool_name, tool_args)
                    print(f"   📊 Result: {result}")
                    
                    trace.append({"tool": tool_name, "args": tool_args, "result": result})
                    messages.append({"role": "tool", "tool_call_id": tc.id, "content": json.dumps(result)})
            else:
                print(f"\n   ✅ Investigation complete")
                return {"recommendation": msg.content, "trace": trace, "iterations": i + 1}
        
        return {"recommendation": "Max iterations reached", "trace": trace}
    
    except Exception as e:
        print(f"\n   ⚠️ API Error: {e}")
        print("   Falling back to demo mode...")
        return run_demo_investigation(alert)

In [ ]:
# Test the agent
alert = """FRAUD ALERT:
- Customer ID: C-789
- Transaction: Wire transfer $45,000 to overseas account
- Flagged by: Amount threshold rule
- Customer tier: Standard (typical transactions under $10K)"""

result = agent_fraud_investigation(alert)

print("\n" + "="*60)
print("FINAL RECOMMENDATION:")
print("="*60)
print(result["recommendation"])

## Part 3: Compare the Approaches

| Aspect | Workflow | Agent |
|--------|----------|-------|
| **Decision maker** | Code (if/else) | LLM |
| **Predictability** | 100% deterministic | Variable |
| **Latency** | Milliseconds | Seconds |
| **Cost** | Near zero | $0.01-0.10 per run |
| **Handles novel cases** | No | Yes |
| **Auditability** | Simple trace | Requires logging |
| **Best for** | Well-defined rules | Complex reasoning |

## 🎯 Exercise: Classify These Banking Processes

For each process, decide: **WORKFLOW**, **AGENT**, or **HYBRID**?

In [ ]:
processes = [
    "ATM cash withdrawal",
    "Investigating a $50K wire flagged as suspicious",
    "Monthly statement generation",
    "Customer asking 'Why was my card declined?'",
    "Processing a mortgage application",
    "Real-time fraud scoring for card transactions",
    "Responding to a regulatory audit request",
    "Calculating daily interest accrual"
]

# Fill in your answers
your_answers = ["", "", "", "", "", "", "", ""]

correct = ["WORKFLOW", "AGENT", "WORKFLOW", "HYBRID", "HYBRID", "WORKFLOW", "AGENT", "WORKFLOW"]

print("Process Classification Exercise")
print("="*60)
for i, (proc, ans) in enumerate(zip(processes, correct)):
    yours = your_answers[i] if your_answers[i] else "(blank)"
    mark = "✅" if yours.upper() == ans else "❌"
    print(f"{mark} {proc}")
    print(f"   Your answer: {yours} | Correct: {ans}")

### 💡 Answer Explanations

| Process | Answer | Reasoning |
|---------|--------|----------|
| ATM withdrawal | WORKFLOW | Fixed rules: check PIN, balance, dispense |
| $50K wire investigation | AGENT | Requires reasoning about multiple signals |
| Statement generation | WORKFLOW | Deterministic aggregation and formatting |
| Card decline explanation | HYBRID | Workflow finds reason, agent explains naturally |
| Mortgage application | HYBRID | Workflow for math, agent for document review |
| Real-time fraud scoring | WORKFLOW | Must be <100ms, use ML model not LLM |
| Regulatory audit | AGENT | Understanding questions, gathering evidence |
| Interest calculation | WORKFLOW | Pure math, no judgment needed |

---
## ✅ Lab 1 Complete!

**Key Takeaways:**
- **Workflows** are deterministic, fast, cheap - use for well-defined processes
- **Agents** are flexible, can reason - use when judgment is needed
- Most production systems are **HYBRID** - workflow structure with agent decision points

**Next:** Open `02_agent_taxonomy.ipynb`